<a href="https://www.kaggle.com/code/charlesguthmann/chuck-first-notebook?scriptVersionId=111674996" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.info()
#super useful for quickly seeing which columns have null data, for instance
#if we want a less beautiful, more simple version, train_data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
num_data = train_data._get_numeric_data().columns
num_data

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [6]:
train_data["Sex"] = train_data["Sex"].astype("category")
train_data["Sex"] = train_data["Sex"].cat.codes
train_data["Sex"].describe()
#64% of the titanic was men
# man == 1

##  _______________________________test data _______________________________________________

test_data["Sex"] = test_data["Sex"].astype("category")
test_data["Sex"] = test_data["Sex"].cat.codes

# alternatively, use map function

In [7]:
#make categorical vars
train_data["Pclass"].unique()

train_data["class2"] = [1 if x == 2 else 0 for x in train_data["Pclass"]]
train_data["class3"] = [1 if x == 3 else 0 for x in train_data["Pclass"]]

##  _______________________________test data _______________________________________________

test_data["class2"] = [1 if x == 2 else 0 for x in test_data["Pclass"]]
test_data["class3"] = [1 if x == 3 else 0 for x in test_data["Pclass"]]

In [8]:
#Now we want to fill in the embarked - 
train_data.describe(include=['O'])
#first, its include = O, like the letter o for object. Second, S the most popular, so we will fill that in
train_data["Embarked"].fillna("S", inplace=True)


In [9]:
#change embarked to numbers using map function

train_data["Embarked"].unique() 
#"S", 'C', 'Q'
train_data["Embarked"]= train_data["Embarked"].map({"S":0, "C":1, "Q":2})

print('_'*40)


test_data["Embarked"]= test_data["Embarked"].map({"S":0, "C":1, "Q":2})

________________________________________


In [10]:
train_data = train_data.drop(columns = ["Ticket", "Cabin", "Name"])

In [11]:
train_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,class2,class3
0,1,0,3,1,22.0,1,0,7.2500,0,0,1
1,2,1,1,0,38.0,1,0,71.2833,1,0,0
2,3,1,3,0,26.0,0,0,7.9250,0,0,1
3,4,1,1,0,35.0,1,0,53.1000,0,0,0
4,5,0,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,0,1,0
887,888,1,1,0,19.0,0,0,30.0000,0,0,0
888,889,0,3,0,NaN,1,2,23.4500,0,0,1
889,890,1,1,1,26.0,0,0,30.0000,1,0,0


In [12]:
## Test data has a single missing fair price, let's fill it with the average. 
test_data.isna().sum()
test_data["Fare"].fillna(14, inplace=True)

In [13]:
#Columns for lin reggresion

independent_vars = ["Sex", "SibSp", "Parch", "Fare", "class2", "class3"]

dependent_var = ["Survived"]

X_train = train_data[independent_vars]
Y_train = train_data[dependent_var]

X_test = test_data[independent_vars]

## Linear Regression

In [14]:
linear_model = LinearRegression()
linear_model.fit(train_data[independent_vars], train_data[dependent_var])
linear_model.predict(test_data[independent_vars]) #if >.5 -> round to 1, else -> round to 0
results = np.round(np.clip(linear_model.predict(test_data[independent_vars]), 0, 1))

## .77 accuracy

## Random Forrest

In [15]:
independent_vars = ["Sex", "SibSp", "Parch", "Fare", "Pclass", "Embarked"]

dependent_var = ["Survived"]

X_train = train_data[independent_vars]
Y_train = train_data[dependent_var]

X_test = test_data[independent_vars]

In [16]:
rf_model = RandomForestClassifier(n_estimators = 100)
rf_model.fit(X_train, Y_train.values.ravel())

RandomForestClassifier()

In [17]:
Y_pred = rf_model.predict(X_test)
rf_pred = list(Y_pred)


rf_submission = test_data[["PassengerId", "Pclass"]]
rf_submission["Survived"] = rf_pred

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
rf_submission = rf_submission.drop("Pclass", axis = 1)
rf_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [19]:
rf_submission.to_csv("rf_submission_monday.csv", index = False)